In [1]:
import boto3
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO
from kiner.producer import KinesisProducer




In [2]:
import boto3
import json
from datetime import datetime
import calendar
import random
import time

my_stream_name = 'bun-ad-prediction-test'

kinesis_client = boto3.client('kinesis', region_name='ap-northeast-2')

# def put_to_stream(thing_id, property_value, property_timestamp):
#     payload = {
#                 'prop': str(property_value),
#                 'timestamp': str(property_timestamp),
#                 'thing_id': thing_id
#               }

#     print payload

#     put_response = kinesis_client.put_record(
#                         StreamName=my_stream_name,
#                         Data=json.dumps(payload)
#                         PartitionKey=thing_id)

# while True:
#     property_value = random.randint(40, 120)
#     property_timestamp = calendar.timegm(datetime.utcnow().timetuple())
#     thing_id = 'aa-bb'

#     put_to_stream(thing_id, property_value, property_timestamp)

#     # wait for 5 second
#     time.sleep(5)


In [7]:
q = """
SELECT UID,
       pid,
       pu_id
FROM service1_quicket.ad_power_up
WHERE to_char(created_at, 'YYYY/MM/dd') = '2020/11/10';
"""

pu = pd.read_sql(q, con=bun_dw)

In [8]:
pu.head()

,uid,pid,pu_id
0,4562617,112950381,119548
1,8574827,98807536,119552
2,13002636,137284116,119556
3,12851816,137073840,119560
4,1624158,137060714,119564


In [9]:
pu_list = pu.groupby('uid')['pid'].apply(list).reset_index()

In [10]:
pu_list

,uid,pid
0,66620,"[133959482, 131351152, 132419409, 132419409, 1..."
1,269655,[137494204]
2,444006,[133825274]
3,518687,[121455293]
4,592959,"[137629625, 88067819, 75300132, 75300132]"
...,...,...
118,13002016,[137279450]
119,13002636,[137284116]
120,13017800,[137603979]
121,13052103,[137638544]


In [3]:
def build_json(uid, adSetProducts):
    return {
   "data":{
      "adSet":{
         "id":80,
         "uid":uid,
         "name":"광고 세트 49",
         "status":"IN_REVIEW",
         "targetGender":"WOMAN",
         "spentType":"FAST_PUBLISHED",
         "adType":"SA",
         "payType":"CPC",
         "totalBudget":20000,
         "paidBudget":0,
         "freeBudget":20000,
         "totalPoint":0,
         "paidPoint":0,
         "freePoint":0,
         "creatorUid":9145992,
         "modifierUid":9145992,
         "dailyBudget":[
            {
               "startDateTime":"2020-11-21T19:00:00",
               "endDateTime":"2020-11-21T23:59:59",
               "paidBudget":0,
               "freeBudget":478,
               "paidPoint":0,
               "freePoint":0
            }
         ],
         "startDateTime":"2020-11-21T19:00:00",
         "endDateTime":"2020-11-30T11:59:59",
         "targetAddressList":[
            3,
            7,
            8
         ],
         "targetAges":[
            {
               "minAge":11,
               "maxAge":20
            },
            {
               "minAge":31,
               "maxAge":40
            }
         ],
         "adSetProducts":adSetProducts
      },
      "before":{
         
      }
   },
   "eventType":"ADSET_REGISTERED",
   "eventTime":"2020-11-06T12:28:25.398",
   "actorUid":9145992
}

In [12]:
for idx, row in pu_list.iterrows():
    uid = row['uid']
    pids = row['pid']
    adSetProducts = []
    for pid in pids:
        adSetProducts.append(
            {
                "adSetProductId": 1,
                "pid": pid,
                "status": "IN_PROGRESS"
            }
        )
    adSet = build_json(uid, adSetProducts)
    print(adSet)
    kinesis_client.put_record(
        StreamName = my_stream_name,
        Data = json.dumps(adSet),
        PartitionKey = 'tracker_test'
    )

{'data': {'adSet': {'id': 80, 'uid': 66620, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 132419409, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 133959584, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 133959584, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 133959482, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 13241

{'data': {'adSet': {'id': 80, 'uid': 1118219, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 129116958, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 1263779, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spen

{'data': {'adSet': {'id': 80, 'uid': 2597771, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 57895412, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 2611849, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spent

{'data': {'adSet': {'id': 80, 'uid': 4049935, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 136059529, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 98901192, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 4054315, 'name': 

{'data': {'adSet': {'id': 80, 'uid': 4644901, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 134654540, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 119655636, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 137227240, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUi

{'data': {'adSet': {'id': 80, 'uid': 5291978, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 136653051, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 5346440, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spen

{'data': {'adSet': {'id': 80, 'uid': 8574827, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 98807536, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 98852066, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 9379199, 'name': '

{'data': {'adSet': {'id': 80, 'uid': 11610698, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 137595126, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 11610922, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'sp

{'data': {'adSet': {'id': 80, 'uid': 12245856, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 137607825, 'status': 'IN_PROGRESS'}, {'adSetProductId': 1, 'pid': 137606354, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 12278034, 'name

{'data': {'adSet': {'id': 80, 'uid': 13052103, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'spentType': 'FAST_PUBLISHED', 'adType': 'SA', 'payType': 'CPC', 'totalBudget': 20000, 'paidBudget': 0, 'freeBudget': 20000, 'totalPoint': 0, 'paidPoint': 0, 'freePoint': 0, 'creatorUid': 9145992, 'modifierUid': 9145992, 'dailyBudget': [{'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-21T23:59:59', 'paidBudget': 0, 'freeBudget': 478, 'paidPoint': 0, 'freePoint': 0}], 'startDateTime': '2020-11-21T19:00:00', 'endDateTime': '2020-11-30T11:59:59', 'targetAddressList': [3, 7, 8], 'targetAges': [{'minAge': 11, 'maxAge': 20}, {'minAge': 31, 'maxAge': 40}], 'adSetProducts': [{'adSetProductId': 1, 'pid': 137638544, 'status': 'IN_PROGRESS'}]}, 'before': {}}, 'eventType': 'ADSET_REGISTERED', 'eventTime': '2020-11-06T12:28:25.398', 'actorUid': 9145992}
{'data': {'adSet': {'id': 80, 'uid': 13054189, 'name': '광고 세트 49', 'status': 'IN_REVIEW', 'targetGender': 'WOMAN', 'sp

In [11]:
# -*- coding: utf-8 -*-
import boto3
import json
kinesis_client = boto3.client('kinesis', region_name='ap-northeast-2')
stream_name = 'bun-ad-prediction-test'
def put_to_stream(thing_id, payload):
    put_response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=json.dumps(payload),
        PartitionKey=thing_id)
    print(put_response)
if __name__ == "__main__":
    thing_id = 'foo'
    payload = {
        "data": {
            "adSet": {
                "id": 80,
                "uid": 9145992,
                "name": "광고 세트 49",
                "status": "IN_REVIEW",
                "targetGender": "WOMAN",
                "spentType": "FAST_PUBLISHED",
                "adType": "SA",
                "payType": "CPC",
                "totalBudget": 20000,
                "paidBudget": 0,
                "freeBudget": 20000,
                "totalPoint": 0,
                "paidPoint": 0,
                "freePoint": 0,
                "creatorUid": 9145992,
                "modifierUid": 9145992,
                "dailyBudget": [
                    {
                        "startDateTime": "2020-11-21T19:00:00",
                        "endDateTime": "2020-11-21T23:59:59",
                        "paidBudget": 0,
                        "freeBudget": 478,
                        "paidPoint": 0,
                        "freePoint": 0
                    }
                ],
                "startDateTime": "2020-11-21T19:00:00",
                "endDateTime": "2020-11-30T11:59:59",
                "targetAddressList": [
                    3,
                    7,
                    8
                ],
                "targetAges": [
                    {
                        "minAge": 11,
                        "maxAge": 20
                    },
                    {
                        "minAge": 31,
                        "maxAge": 40
                    }
                ],
                "adSetProducts": [
                    {
                        "adSetProductId": 159,
                        "pid": 2000001435,
                        "status": "IN_REVIEW",
                        "keywords": [
                            {
                                "keyword": "아이폰",
                                "bidPrice": 120,
                            }
                        ]
                    },
                    {
                        "adSetProductId": 160,
                        "pid": 2000001436,
                        "status": "IN_REVIEW"
                    }
                ]
            },
            "before": {
            }
        },
        "eventType": "ADSET_REGISTERED",
        "eventTime": "2020-11-06T12:28:25.398",
        "actorUid": 9145992
    }
    put_to_stream(thing_id, payload)

{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49612546023161128734163179899385137168973777711795273730', 'ResponseMetadata': {'RequestId': 'c9a412f4-aa1e-eee5-9188-0ac9c78c927f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c9a412f4-aa1e-eee5-9188-0ac9c78c927f', 'x-amz-id-2': 'odVAhzONAILwRKJh4MOrS7dCvVDuHUig+kUrEhOweXk5TrZk1ezuAEo6n9YTpuX6h0HryV+z0f5ZVB0KgzzAo+tez8R24CoC', 'date': 'Mon, 16 Nov 2020 06:02:37 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}}
